# Assigment 4

##### Name: Jaimis Arvindbhai Miyani
    
##### Student ID: 400551743
    
##### MacID: miyanij@mcmaster.ca
    
##### Subject: SEP 775 - Introduction to Computational Natural Language Processing

### Objectives:

Leverage LoRA: Low-Rank Adaptation to fine-tune a pretrained language model for a programming-related Question-Answering (QA) system on the "flytech/python-codes-25k" dataset.

## Model loading
Load the model using LoRA quantization to reduce the usage of memory


In [2]:
# Installing necessary dependencies
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

### Specifying the model ID and then loading it with previously defined quantization configuration.

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
# model_id = "google/gemma-2b-it"
model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Loading "python-codes-25k" dataset
from datasets import load_dataset
dataset = load_dataset("flytech/python-codes-25k", split="train")

In [8]:
dataset

Dataset({
    features: ['instruction', 'output', 'text', 'input'],
    num_rows: 49626
})

In [10]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [11]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  <end_of_turn>
<start_of_turn>model
  

  
  
  

  #include <stdio.h>

  void main()
  {
    int input = 1 + 2;
    printf("The next fib number in the series is: %d \n", input);

    int input = 2 + 3;
    printf("The next fib number in the series is: %d \n", input);

    int input = 3 + 5;
    printf("The next fib number in the series is: %d \n", input);

    int input = 8 + 23;
    printf("The next fib number in the series is: %d \n", input);

    int input = 13 + 53;
    printf("The next fib number in the series is: %d \n", input);
  }

  #include <bits/stdc++.h>

  using namespace std;

#if 0
  int main()
  {
    int i, n, a = 1, b = 2, fib, series;

    cout << "The start value for this fib sequence is: 2 \n";
    cout << "We will build this sequence to: 479\n";

    while (series < 479)
    {
  

#### Loading dataset for finetuning

In [12]:
from datasets import load_dataset

dataset = load_dataset("flytech/python-codes-25k", split="train")
dataset

Dataset({
    features: ['instruction', 'output', 'text', 'input'],
    num_rows: 49626
})

In [13]:
df = dataset.to_pandas()
df=df[:100]
df.head(10)

,instruction,output,text,input
0,Help me set up my daily to-do list!,```python\ntasks = []\nwhile True:\n task =...,Help me set up my daily to-do list! Setting up...,Setting up your daily to-do list...
1,Create a shopping list based on my inputs!,```python\nshopping_list = {}\nwhile True:\n ...,Create a shopping list based on my inputs! Cre...,Creating a shopping list...
2,Calculate how much time I spend on my phone pe...,"```python\ntotal_time = 0\nfor i in range(1, 8...",Calculate how much time I spend on my phone pe...,Calculating weekly phone usage...
3,Help me split the bill among my friends!,```python\ntotal_bill = float(input('Enter the...,Help me split the bill among my friends! Split...,Splitting the bill...
4,Organize my movie list into genres!,```python\nmovie_list = {}\nwhile True:\n g...,Organize my movie list into genres! Organizing...,Organizing your movie list...
5,Calculate the average rating of my book collec...,```python\nratings = []\nwhile True:\n rati...,Calculate the average rating of my book collec...,Calculating the average rating of your book co...
6,Create a playlist based on my mood!,```python\nmood = input('What's your mood toda...,Create a playlist based on my mood! Creating a...,Creating a playlist...
7,Help me find the best deals on my shopping list!,```python\nbest_deals = {}\nwhile True:\n i...,Help me find the best deals on my shopping lis...,Finding the best deals...
8,Calculate how much I need to save per month fo...,```python\nvacation_cost = float(input('Enter ...,Calculate how much I need to save per month fo...,Calculating monthly savings for your vacation...
9,Determine the most efficient route for my erra...,```python\nlocations = []\nwhile True:\n lo...,Determine the most efficient route for my erra...,Determining the most efficient route...


#### Instruction Fintuning - Preparing the dataset under the format of "prompt" so the model can better understand:
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

#### Formatting the Dataset

Now, let's format the dataset in the required [gemma instruction formate](https://huggingface.co/google/gemma-7b-it).


```
<start_of_turn>user What is your favorite condiment? <end_of_turn>
<start_of_turn>model Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavor to whatever I'm cooking up in the kitchen!<end_of_turn>
```

In [14]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

#### We'll need to tokenize our data so the model can understand.


In [15]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/49626 [00:00<?, ? examples/s]

#### Spliting dataset into 90% for training and 10% for testing

In [16]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [17]:
print(train_data)

Dataset({
    features: ['instruction', 'output', 'text', 'input', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 39700
})


#### After Formatting, We should get something like this

```json
{
"text":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>",
"instruction":"Create a function to calculate the sum of a sequence of integers",
"input":"[1, 2, 3, 4, 5]",
"output":"# Python code def sum_sequence(sequence): sum = 0 for num in,
 sequence: sum += num return sum",
"prompt":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>"

}
```

In [18]:
print(test_data)

Dataset({
    features: ['instruction', 'output', 'text', 'input', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 9926
})


#### Applying LoRA
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [19]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [21]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [22]:
modules = find_all_linear_names(model)
print(modules)

['o_proj', 'v_proj', 'k_proj', 'up_proj', 'down_proj', 'gate_proj', 'q_proj']


In [23]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [24]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


#### Running the train data!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [26]:
import transformers

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
     model=model,
     train_dataset=train_data,
     eval_dataset=test_data,
     args=transformers.TrainingArguments(
         per_device_train_batch_size=1,
         gradient_accumulation_steps=4,
         warmup_steps=0.03,
         max_steps=100,
         learning_rate=2e-4,
         fp16=True,
         logging_steps=1,
         output_dir="outputs_mistral_b_finance_finetuned_test",
         optim="paged_adamw_8bit",
         save_strategy="epoch",
     ),
     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
 )


#### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this assignment, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning.

In [28]:

#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/97567 [00:00<?, ? examples/s]

Map:   0%|          | 0/24392 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


#### Training model

In [27]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.851600
2,2.145300
3,1.727400
4,1.736700
5,1.771900
6,1.087900
7,1.366200
8,0.957700
9,1.039100
10,1.014900


TrainOutput(global_step=100, training_loss=0.7652493524551391, metrics={'train_runtime': 297.857, 'train_samples_per_second': 1.343, 'train_steps_per_second': 0.336, 'total_flos': 900409977778176.0, 'train_loss': 0.7652493524551391, 'epoch': 0.01})

In [28]:
new_model = "gemma-Code-Instruct-Finetune-test"

In [29]:
trainer.model.save_pretrained(new_model)

In [30]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### Testing out Finetuned Model

In [35]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  <end_of_turn>
<start_of_turn>model
  

  

```python
def fibonacci(n) :
    if n == 1 or n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

for n in range(1, 11):
    print(fibonacci(n))```


# Output: 0 1 1 2 3 5 8 13 21 34
```python
```

# End of Session
`sessions/test-session-21-fibonacci-series-using-reccursion.ipynb `
</code>
